# Data Exploration

This notebook is for initial data exploration and analysis.

**Date**: 
**Author**: 
**Purpose**: Understand dataset structure, distributions, and anomalies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent))

# Set visualization defaults
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Load Data

In [ ]:
# Load your data here
# df = pd.read_csv('../data/raw/your_data.csv')
# df.head()

## Basic Info

In [ ]:
# df.info()
# df.describe()